In [1]:
import os
import datetime
from pathlib import Path

def collect_project_info(root_path, output_file="project_complete_info.txt"):
    """
    Thu thập cấu trúc thư mục và nội dung tất cả file code
    """
    # Các extension file code phổ biến
    code_extensions = {
        '.py', '.js', '.css', '.json', '.xml', '.yml', '.yaml',
        '.md', '.txt', '.sql', '.sh', '.bat', '.cfg', '.ini', '.env',
        '.jsx', '.ts', '.tsx', '.vue', '.php', '.java', '.cpp', '.c',
        '.h', '.cs', '.rb', '.go', '.rs', '.swift', '.kt'
    }
    
    # Thư mục và file cần bỏ qua
    ignore_dirs = {
        '__pycache__', '.git', '.vscode', '.idea', 'node_modules', 
        '.pytest_cache', '.mypy_cache', 'venv', 'env', '.env',
        'dist', 'build', '.next', '.nuxt'
    }
    
    ignore_files = {
        '.pyc', '.pyo', '.pyd', '.so', '.dll', '.exe', '.log',
        '.tmp', '.temp', '.cache', '.DS_Store', 'Thumbs.db'
    }
    
    # THÊM: Danh sách file cụ thể cần bỏ qua
    ignore_specific_files = {
        'employees.json', 'depots.json', 'transport_providers.json', 'main.py'
    }
    
    root_path = Path(root_path)
    
    # SỬA LỖI: Đảm bảo output_file là string
    if isinstance(output_file, Path):
        output_file = str(output_file)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        # Header thông tin
        f.write("=" * 80 + "\n")
        f.write("THÔNG TIN DỰ ÁN HOÀN CHỈNH\n")
        f.write("=" * 80 + "\n")
        f.write(f"Thời gian tạo: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Thư mục gốc: {root_path.absolute()}\n")
        f.write("=" * 80 + "\n\n")
        
        # 1. Cấu trúc thư mục
        f.write("📁 CẤU TRÚC THU MỤC\n")
        f.write("-" * 50 + "\n")
        write_directory_tree(f, root_path, ignore_dirs, ignore_specific_files)
        f.write("\n\n")
        
        # 2. Danh sách tất cả file
        f.write("📄 DANH SÁCH TẤT CẢ FILE\n")
        f.write("-" * 50 + "\n")
        all_files = list_all_files(root_path, ignore_dirs, ignore_files, ignore_specific_files)
        for file_path in sorted(all_files):
            rel_path = file_path.relative_to(root_path)
            file_size = file_path.stat().st_size
            f.write(f"{rel_path} ({file_size} bytes)\n")
        f.write(f"\nTổng cộng: {len(all_files)} file\n\n")
        
        # 3. Nội dung tất cả file code
        f.write("💻 NỘI DUNG TẤT CẢ FILE CODE\n")
        f.write("=" * 80 + "\n")
        
        code_files = [f for f in all_files if f.suffix.lower() in code_extensions]
        
        for file_path in sorted(code_files):
            rel_path = file_path.relative_to(root_path)
            f.write(f"\n{'='*60}\n")
            f.write(f"FILE: {rel_path}\n")
            f.write(f"{'='*60}\n")
            
            try:
                # Thử đọc với UTF-8 trước
                content = file_path.read_text(encoding='utf-8')
                f.write(content)
            except UnicodeDecodeError:
                try:
                    # Nếu không được thì thử với encoding khác
                    content = file_path.read_text(encoding='latin-1')
                    f.write(content)
                except Exception as e:
                    f.write(f"[KHÔNG THỂ ĐỌC FILE: {e}]\n")
            except Exception as e:
                f.write(f"[LỖI ĐỌC FILE: {e}]\n")
            
            f.write(f"\n{'='*60}\n")
        
        # 4. Thống kê cuối
        f.write(f"\n\n📊 THỐNG KÊ\n")
        f.write("-" * 30 + "\n")
        f.write(f"Tổng số file: {len(all_files)}\n")
        f.write(f"File code: {len(code_files)}\n")
        f.write(f"Các loại file code:\n")
        
        extensions = {}
        for file in code_files:
            ext = file.suffix.lower()
            extensions[ext] = extensions.get(ext, 0) + 1
        
        for ext, count in sorted(extensions.items()):
            f.write(f"  {ext}: {count} file\n")

def write_directory_tree(f, path, ignore_dirs, ignore_specific_files, prefix=""):
    """Viết cấu trúc thư mục dạng tree"""
    try:
        items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name.lower()))
        
        for i, item in enumerate(items):
            if item.name.startswith('.') and item.name not in {'.env', '.gitignore'}:
                continue
            if item.name in ignore_dirs:
                continue
            # THÊM: Bỏ qua file cụ thể
            if item.name in ignore_specific_files:
                continue
                
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            
            if item.is_dir():
                f.write(f"{prefix}{current_prefix}📁 {item.name}/\n")
                next_prefix = prefix + ("    " if is_last else "│   ")
                write_directory_tree(f, item, ignore_dirs, ignore_specific_files, next_prefix)
            else:
                f.write(f"{prefix}{current_prefix}📄 {item.name}\n")
                
    except PermissionError:
        f.write(f"{prefix}[KHÔNG CÓ QUYỀN TRUY CẬP]\n")

def list_all_files(root_path, ignore_dirs, ignore_files, ignore_specific_files):
    """Lấy danh sách tất cả file"""
    all_files = []
    
    for item in root_path.rglob("*"):
        # Bỏ qua thư mục ignore
        if any(ignore_dir in item.parts for ignore_dir in ignore_dirs):
            continue
            
        # Bỏ qua file ẩn (trừ một số file quan trọng)
        if item.name.startswith('.') and item.name not in {'.env', '.gitignore', '.htaccess'}:
            continue
            
        # Bỏ qua file theo extension
        if item.suffix.lower() in ignore_files:
            continue
        
        # THÊM: Bỏ qua file cụ thể theo tên
        if item.name in ignore_specific_files:
            continue
            
        if item.is_file():
            all_files.append(item)
    
    return all_files

# Sử dụng script
if __name__ == "__main__":
    # Thay đổi đường dẫn này thành thư mục dự án của bạn
    project_path = "."  # Thư mục hiện tại
    output_file = "project_complete_info.txt"  # Đảm bảo là string
    
    print("🔄 Đang thu thập thông tin dự án...")
    collect_project_info(project_path, output_file)
    print(f"✅ Hoàn thành! Thông tin đã được lưu vào: {output_file}")
    print(f"📁 File size: {os.path.getsize(output_file)} bytes")

🔄 Đang thu thập thông tin dự án...
✅ Hoàn thành! Thông tin đã được lưu vào: project_complete_info.txt
📁 File size: 414060 bytes


In [2]:
# import requests
# import json

# # Thay bằng URL webhook của bot bạn tạo trong group Lark
# webhook_url = "https://open.larksuite.com/open-apis/bot/v2/hook/6a53a060-40d7-4716-9a90-970a6cbdaf64"

# # Nội dung tin nhắn muốn gửi
# message_data = {
#     "msg_type": "text",
#     "content": {
#         "text": "Xin chào! Đây là tin nhắn test gửi vào nhóm Lark bằng API."
#     }
# }

# headers = {
#     "Content-Type": "application/json"
# }

# response = requests.post(webhook_url, headers=headers, data=json.dumps(message_data))

# if response.status_code == 200:
#     print("Gửi tin nhắn thành công!")
# else:
#     print(f"Lỗi khi gửi tin nhắn: {response.status_code}, {response.text}")
